### Installation

In [ ]:
!pip install -q langchain sentence-transformers cohere

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install rank_bm25

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [ ]:
import os
from getpass import getpass

# HF_token = getpass()

In [ ]:
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

In [ ]:
dataset_folder_path='/content/drive/MyDrive/RESUMES/resume_dataset (1)'

In [ ]:
!pip install pypdf

In [ ]:
# documents=[]
from langchain_community.document_loaders import PyPDFLoader
# file_count = 0
# max_files = 100
# # from PyPDF2 import PdfFileReader,PdfReader
# for file in os.listdir(dataset_folder_path):
#     # if file_count >= max_files:
#     #     break
#     file_path = os.path.join(dataset_folder_path, file)
#     loader = PyPDFLoader(file_path)
#     documents.extend(loader.load())
#     # file_count += 1


documents = []
from langchain_community.document_loaders import PyPDFLoader
import os

# dataset_folder_path = '/path/to/your/dataset/folder'
max_files = 100
file_count = 0

for file in os.listdir(dataset_folder_path):
    file_path = os.path.join(dataset_folder_path, file)

    # Check if the file is not empty
    if os.path.getsize(file_path) == 0:
        print(f"Skipping empty file: {file}")
        continue

    loader = PyPDFLoader(file_path)
    try:
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())

    except :
        print(f"Error loading file {file}")
        continue

    # file_count += 1
    # if file_count >= max_files:
    #     break

Skipping empty file: sandhyalakshmi676@gmail.com_resume.pdf


In [ ]:
len(documents)
# print(documents[156])

1518

In [ ]:
documents[:3]

[Document(metadata={'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/aks76625@gmail.com_resume.pdf', 'page': 0}, page_content='Anurag Shukla\nak s76625@gmail.c om | +91 95110 7 5594\nCAREER GOALS\nTopursue ajobopportunity inacompetitive\nenvironment that willchallenge metopush\nmyboundaries andexpand myknowledge in\nthefield ofcomputer science while allowing\nmetoaddvalue tothedynamics ofthecoun-\ntry.\nEDUCATION\nINSTITUTE OF ENGINEERING AND\nTECHNOLOGY, DAVV\nB.E. IN INF ORMA TION TECHNOL OG Y\nA ug 2019 - Ma y 2023 | Indore, M.P\nCGP A: 8.32/ 10\nBETHANY CONVENT SCHOOL\nINTERMEDIA TE (PCM)\nMa y 2018 | ALLAHAB AD, U .P\nPERCENT AGE: 92.4\nDELHI PUBLIC SCHOOL\nHIGH SCHOOL\nMa y 2016 | V ADOD AR A , GU J AR A T\nCGP A: 10 / 10\nLINKS\nPortfolio: Anurag Shukla\nGithub: aks1817\nLinkedIn: Anurag Shukla\nInterviewBit: Anurag Shukla\nLeetCode: aks76625\nGeeksforGeeks: aks76625\nHackerRank: Anurag_1817\nCOURSEWORK\nUNDERGRADUATE\nDatabase Management Systems\nOperating Systems\n

In [ ]:
len(documents)

1518

### Chunking the text

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=50)
text_splits=text_splitter.split_documents(documents)
print(len(text_splits))

9341


### Embedding Model

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a light and faster experience.
    # model_kwargs={'device':'cpu'},
    # encode_kwargs={'normalize_embeddings': True}
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when p

In [ ]:
vectorstore = FAISS.from_documents(text_splits, embeddings)

In [ ]:
vectorstore.save_local("embedding1000")

## **Implementing Hybrid Search with ensemble Retrieval**

> Indented block



In [ ]:
retriever_vectordb = vectorstore.as_retriever(search_kwargs={"k": 20})

In [ ]:
keyword_retriever = BM25Retriever.from_documents(text_splits)
keyword_retriever.k =  10



In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],
                                       weights=[0.5, 0.5])

In [ ]:
query="machine learning engineer with 5 year experience"



In [ ]:
docs_and_scores = retriever_vectordb.invoke(query)

In [ ]:
print(docs_and_scores)

[Document(metadata={'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/bhatiatushar02@gmail.com_resume.pdf', 'page': 0}, page_content='learning techniques and algorithms for accurate identification and assessment.\n◦Tech Stack : Python, Tensorflow, OpenCV'), Document(metadata={'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/utpalraj040503@gmail.com_resume.pdf', 'page': 0}, page_content='Natural Language Processing in TensorFlow  by DeepLearning.AI | StatPhysics  India Conference and Summer School | AI Intern at Cloud Counselling  \n Machine  Learning  Specialization  by Stanford  University  and DeepLearning.AI  | Supervised  Machine  Learning:  Regression  and Classification  | Advanced  Learning \nAlgorithms  | Unsupervised Learning : Recommenders, Reinforcement  Learning  | Data Science Intern at YBI Foundation'), Document(metadata={'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/chanderaravi1005@gmail.com_resume.pdf', 'page': 0}, page_content='Sk

In [ ]:
docs_rel=ensemble_retriever.get_relevant_documents(query)
docs_rel


[Document(metadata={'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/bhatiatushar02@gmail.com_resume.pdf', 'page': 0}, page_content='learning techniques and algorithms for accurate identification and assessment.\n◦Tech Stack : Python, Tensorflow, OpenCV'),
 Document(metadata={'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/gbr8214@gmail.com_resume.pdf', 'page': 0}, page_content='Implemented and evaluated artificial intelligence and machine learning algorithms and neural networks for diverse \nindustries. •\nPrototyped machine learning applications and quickly determined application viability. •\nCreated customized applications to be used for critical predictions, automated reasoning and decisions, and \ncalculated optimization algorithms. •\nResearched, designed, and implemented machine learning applications to solve business problems affecting many \nusers.•\nPROJECTS'),
 Document(metadata={'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/utpalraj0

In [ ]:
for doc in docs_rel:
    x = doc.metadata
    print(x)
unique_docs = list(set(x))
print(unique_docs)

{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/bhatiatushar02@gmail.com_resume.pdf', 'page': 0}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/gbr8214@gmail.com_resume.pdf', 'page': 0}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/utpalraj040503@gmail.com_resume.pdf', 'page': 0}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/bhadani.neeraj.08@gmail.com_resume.pdf', 'page': 0}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/chanderaravi1005@gmail.com_resume.pdf', 'page': 0}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/rexonpambujya2001@gmail.com_resume.pdf', 'page': 0}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/chethandanivas@gmail.com_resume.pdf', 'page': 8}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/sureshbhojwanics@gmail.com_resume.pdf', 'page': 0}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/1704346541089_resume.pdf', 'page': 1}
{'source'

### Implementing Re-ranking with Cohere-Rerank

In [ ]:
# Cohere_API_token = getpass()

In [ ]:
import os
os.environ["COHERE_API_KEY"] ="<API>"

In [ ]:
!pip install --upgrade --quiet  cohere

In [ ]:
!pip install langchain-cohere

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere
compressor = CohereRerank(top_n=10)
# top_n=5
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_vectordb
)
compressed_docs = compression_retriever.get_relevant_documents(query)

In [ ]:
compression_retriever

ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x7fae295db3d0>, top_n=10, model='rerank-english-v3.0', cohere_api_key=None, user_agent='langchain:partner'), base_retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fae22ee5e40>, search_kwargs={'k': 10}))

In [ ]:
compressed_docs

[Document(metadata={'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/sachinmr722@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.9856077}, page_content='Sachin M R\nComputer Vision EngineerImmediate Joiner | Machine Learning Engineer with 3.9 years of experience and \na proven track record in developing deep learning models and integrating them \nwith diverse cloud service providers, including AWS. Proficient in optimizing deep \nlearning models and porting applications for Edge devices, using containerized \napplications and orchestrating containers through the AWS cloud or Kubernetes.'),
 Document(metadata={'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/v.gurupraveen123@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.9780936}, page_content='OBJECTIVE  \nSeasoned Machine Learning Engineer with nearly  3 years of extensive experience in developing \nadvanced algorithms and models for ML and NLP applications . Proven track record in data \npreproc

In [ ]:
for doc in compressed_docs:
    metadata = doc.metadata
    print(metadata)
unique_docs = list(set(metadata))
print(unique_docs)

{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/sachinmr722@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.9856077}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/v.gurupraveen123@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.9780936}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/evaan2001@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.88202196}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/sachinmr722@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.33731267}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/navnidhi1210@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.075995214}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/sureshbhojwanics@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.048405956}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/sureshbhojwanics@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.03507868}
{'source': '/

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere
compressor = CohereRerank(top_n=10)
# top_n=5
compression_retriever1 = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)
compressed_docs_bm25 = compression_retriever.get_relevant_documents(query)

In [ ]:
compression_retriever1

ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x7fae295dba90>, top_n=10, model='rerank-english-v3.0', cohere_api_key=None, user_agent='langchain:partner'), base_retriever=EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fae22ee5e40>, search_kwargs={'k': 10}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7fae22b8fe20>, k=10)], weights=[0.5, 0.5]))

In [ ]:
compressed_docs_bm25

[Document(metadata={'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/sachinmr722@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.9856077}, page_content='Sachin M R\nComputer Vision EngineerImmediate Joiner | Machine Learning Engineer with 3.9 years of experience and \na proven track record in developing deep learning models and integrating them \nwith diverse cloud service providers, including AWS. Proficient in optimizing deep \nlearning models and porting applications for Edge devices, using containerized \napplications and orchestrating containers through the AWS cloud or Kubernetes.'),
 Document(metadata={'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/v.gurupraveen123@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.9780936}, page_content='OBJECTIVE  \nSeasoned Machine Learning Engineer with nearly  3 years of extensive experience in developing \nadvanced algorithms and models for ML and NLP applications . Proven track record in data \npreproc

In [ ]:
for doc in compressed_docs_bm25:
    metadata_bm = doc.metadata
    print(metadata_bm)
unique_docs_bm = list(set(metadata_bm))
print(unique_docs_bm)

{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/sachinmr722@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.9856077}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/v.gurupraveen123@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.9780936}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/evaan2001@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.88202196}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/sachinmr722@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.33731267}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/navnidhi1210@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.075995214}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/sureshbhojwanics@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.048405956}
{'source': '/content/drive/MyDrive/RESUMES/resume_dataset (1)/sureshbhojwanics@gmail.com_resume.pdf', 'page': 0, 'relevance_score': 0.03507868}
{'source': '/